# AI Lesson 04a: SQL for Machine Learning (INSTRUCTOR SOLUTIONS)

## Database Applications Development

**Purpose:** Learn SQL to efficiently work with real-world datasets for machine learning

### Why We're Learning SQL in an AI Course

**The Reality:**
- Most ML data doesn't come from CSV files
- Companies store millions/billions of records in databases
- SQL lets you **filter data BEFORE loading** (saves memory!)
- SQL lets you **combine multiple data sources**
- SQL lets you **create new features** through aggregation

**Today's Dataset:** NBA basketball statistics (5 seasons of data)

**What You'll Learn:**
- Connect to SQLite database
- Write SELECT queries to extract data
- Use WHERE to filter records
- Use ORDER BY to sort results
- Use LIMIT to control output size
- Load SQL results into pandas for ML

Let's get started!

### What is SQL??
SQL (Structured **Query** Language) is used to ask questions of a database.
It tells the database what data you want and how to organize it.

### What is SQLite??
SQLite is a lightweight, file-based database system.  The entire database lives in a single `.db` file.  It uses SQL but does not require a server to run.


---

## Part 1: Setup and Database Connection

In [4]:
# Import the libraries we need
import pandas as pd
import sqlite3

# Connect to the NBA database
conn = sqlite3.connect('nba_5seasons.db')

print("✅ Connected to NBA database successfully!")
print("\nThis database contains:")
print("  - Team information (30 NBA teams)")
print("  - Player information (hundreds of players)")
print("  - Team game statistics (thousands of games)")
print("  - Player season statistics (multiple seasons)")

✅ Connected to NBA database successfully!

This database contains:
  - Team information (30 NBA teams)
  - Player information (hundreds of players)
  - Team game statistics (thousands of games)
  - Player season statistics (multiple seasons)


### What Just Happened?

- `import pandas as pd` - Brings in pandas for DataFrame operations
- `import sqlite3` - Brings in SQLite database functionality
- `sqlite3.connect()` - Opens a connection to our database file
- `conn` - This connection object lets us send SQL queries

**Important:** The database file (`nba_5seasons.db`) must be in the same folder as this notebook!

---

## Part 2: Exploring the Database Structure

In [13]:
# Let's see what tables are in our database
query = """
SELECT name 
FROM sqlite_master 
WHERE type='table'
ORDER BY name
"""

tables = pd.read_sql(query, conn)     # Run the SQL query and store the result in a DataFrame
print("Tables in our database:")      # Print a simple text label to the output
display(tables)                       # Show the DataFrame as a formatted table (Jupyter-only)

# sqlite_master is a built-in SQLite system table that stores database metadata
# We query it to see what tables exist in the database (not the table data itself)

Tables in our database:


,name
0,player_season_stats
1,players
2,team_game_stats
3,teams


**Expected Tables:**
- `players` - Player names and IDs
- `teams` - Team information
- `player_season_stats` - Player statistics by season
- `team_game_stats` - Team performance in each game

In [11]:
# Let's look at the structure of the teams table
query = "PRAGMA table_info(teams)"
team_structure = pd.read_sql(query, conn)
print("Teams table structure:")
display(team_structure)

# PRAGMA is a special SQLite command to get table metadata

Teams table structure:


,cid,name,type,notnull,dflt_value,pk
0,0,team_id,INTEGER,0,None,1
1,1,full_name,TEXT,0,None,0
2,2,abbreviation,TEXT,0,None,0
3,3,nickname,TEXT,0,None,0
4,4,city,TEXT,0,None,0
5,5,state,TEXT,0,None,0
6,6,year_founded,INTEGER,0,None,0


---

## Part 3: Your First SQL Query - SELECT and FROM

In [28]:
# Get all columns from the teams table
query = """
SELECT *
FROM teams
"""

all_teams = pd.read_sql(query, conn)
print("All NBA Teams:")
display(all_teams)

print(f"\nTotal teams: {len(all_teams)}")

All NBA Teams:


,team_id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,San Francisco,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970



Total teams: 30


### SQL Breakdown:

```sql
SELECT *          -- * means "all columns"
FROM teams        -- teams is the table name
```

**Translation:** "Give me all columns from the teams table"

In [34]:
# Select only specific columns
query = """
SELECT full_name, city, state, year_founded
FROM teams
"""

team_basics = pd.read_sql(query, conn)
print("Team Basic Information:")
display(team_basics.head(10))

Team Basic Information:


,full_name,city,state,year_founded
0,Atlanta Hawks,Atlanta,Georgia,1949
1,Boston Celtics,Boston,Massachusetts,1946
2,Cleveland Cavaliers,Cleveland,Ohio,1970
3,New Orleans Pelicans,New Orleans,Louisiana,2002
4,Chicago Bulls,Chicago,Illinois,1966
5,Dallas Mavericks,Dallas,Texas,1980
6,Denver Nuggets,Denver,Colorado,1976
7,Golden State Warriors,San Francisco,California,1946
8,Houston Rockets,Houston,Texas,1967
9,Los Angeles Clippers,Los Angeles,California,1970


### Key Point:

- When you specify columns, separate them with commas
- Order matters - columns appear in the order you list them
- **pandas equivalent:** `df[['full_name', 'city', 'state', 'year_founded']]`

---

## Part 4: WHERE Clause - Filtering Data

In [38]:
# Find teams from California
query = """
SELECT full_name, city, state
FROM teams
WHERE state = 'California'
"""

ca_teams = pd.read_sql(query, conn)
print("California NBA Teams:")
display(ca_teams)

print(f"\nNumber of teams in California: {len(ca_teams)}")

California NBA Teams:


,full_name,city,state
0,Golden State Warriors,San Francisco,California
1,Los Angeles Clippers,Los Angeles,California
2,Los Angeles Lakers,Los Angeles,California
3,Sacramento Kings,Sacramento,California



Number of teams in California: 4


### WHERE Clause Rules:

- Filters rows BEFORE returning results
- Text values need **single quotes**: `'California'`
- Use `=` for "equals" (NOT `==` like in Python!)
- **pandas equivalent:** `df[df['state'] == 'California']`

In [41]:
# Find older teams (founded before 1980)
query = """
SELECT full_name, city, year_founded
FROM teams
WHERE year_founded < 1980
"""

older_teams = pd.read_sql(query, conn)
print("Teams Founded Before 1980:")
display(older_teams)

# Numbers don't need quotes!

Teams Founded Before 1980:


,full_name,city,year_founded
0,Atlanta Hawks,Atlanta,1949
1,Boston Celtics,Boston,1946
2,Cleveland Cavaliers,Cleveland,1970
3,Chicago Bulls,Chicago,1966
4,Denver Nuggets,Denver,1976
5,Golden State Warriors,San Francisco,1946
6,Houston Rockets,Houston,1967
7,Los Angeles Clippers,Los Angeles,1970
8,Los Angeles Lakers,Los Angeles,1948
9,Milwaukee Bucks,Milwaukee,1968


### Comparison Operators:

| Operator | Meaning | Example |
|----------|---------|----------|
| `=` | Equal to | `WHERE state = 'Texas'` |
| `!=` or `<>` | Not equal | `WHERE state != 'California'` |
| `>` | Greater than | `WHERE year_founded > 1990` |
| `<` | Less than | `WHERE year_founded < 1980` |
| `>=` | Greater or equal | `WHERE year_founded >= 1995` |
| `<=` | Less or equal | `WHERE year_founded <= 1970` |

In [44]:
# Multiple conditions with AND
query = """
SELECT full_name, city, state, year_founded
FROM teams
WHERE state = 'California' AND year_founded < 1970
"""

result = pd.read_sql(query, conn)
print("Old California Teams:")
display(result)

# Both conditions must be true

Old California Teams:


,full_name,city,state,year_founded
0,Golden State Warriors,San Francisco,California,1946
1,Los Angeles Lakers,Los Angeles,California,1948
2,Sacramento Kings,Sacramento,California,1948


In [46]:
# Multiple conditions with OR
query = """
SELECT full_name, city, state
FROM teams
WHERE state = 'California' OR state = 'Texas'
"""

result = pd.read_sql(query, conn)
print("Teams from California OR Texas:")
display(result)

# At least one condition must be true

Teams from California OR Texas:


,full_name,city,state
0,Dallas Mavericks,Dallas,Texas
1,Golden State Warriors,San Francisco,California
2,Houston Rockets,Houston,Texas
3,Los Angeles Clippers,Los Angeles,California
4,Los Angeles Lakers,Los Angeles,California
5,Sacramento Kings,Sacramento,California
6,San Antonio Spurs,San Antonio,Texas


---

## Part 5: ORDER BY - Sorting Results

In [49]:
# Sort teams by year founded (oldest first)
query = """
SELECT full_name, city, year_founded
FROM teams
ORDER BY year_founded
"""

teams_by_age = pd.read_sql(query, conn)
print("Teams Sorted by Age (Oldest First):")
display(teams_by_age.head(10))

# Default sort order is ascending (low to high)

Teams Sorted by Age (Oldest First):


,full_name,city,year_founded
0,Boston Celtics,Boston,1946
1,Golden State Warriors,San Francisco,1946
2,New York Knicks,New York,1946
3,Los Angeles Lakers,Los Angeles,1948
4,Sacramento Kings,Sacramento,1948
5,Detroit Pistons,Detroit,1948
6,Atlanta Hawks,Atlanta,1949
7,Philadelphia 76ers,Philadelphia,1949
8,Washington Wizards,Washington,1961
9,Chicago Bulls,Chicago,1966


In [51]:
# Sort teams by year founded (newest first)
query = """
SELECT full_name, city, year_founded
FROM teams
ORDER BY year_founded DESC
"""

teams_newest = pd.read_sql(query, conn)
print("Teams Sorted by Age (Newest First):")
display(teams_newest.head(10))

# DESC means descending (high to low)

Teams Sorted by Age (Newest First):


,full_name,city,year_founded
0,New Orleans Pelicans,New Orleans,2002
1,Toronto Raptors,Toronto,1995
2,Memphis Grizzlies,Memphis,1995
3,Minnesota Timberwolves,Minnesota,1989
4,Orlando Magic,Orlando,1989
5,Miami Heat,Miami,1988
6,Charlotte Hornets,Charlotte,1988
7,Dallas Mavericks,Dallas,1980
8,Denver Nuggets,Denver,1976
9,Brooklyn Nets,Brooklyn,1976


### ORDER BY Options:

- `ORDER BY column` - Sort ascending (default)
- `ORDER BY column ASC` - Sort ascending (explicit)
- `ORDER BY column DESC` - Sort descending
- **pandas equivalent:** `df.sort_values('column')`

In [54]:
# Sort by multiple columns
query = """
SELECT full_name, city, state, year_founded
FROM teams
ORDER BY state, year_founded
"""

teams_multi_sort = pd.read_sql(query, conn)
print("Teams Sorted by State, then by Year:")
display(teams_multi_sort.head(15))

# First sorts by state (alphabetically), then by year within each state

Teams Sorted by State, then by Year:


,full_name,city,state,year_founded
0,Phoenix Suns,Phoenix,Arizona,1968
1,Golden State Warriors,San Francisco,California,1946
2,Los Angeles Lakers,Los Angeles,California,1948
3,Sacramento Kings,Sacramento,California,1948
4,Los Angeles Clippers,Los Angeles,California,1970
5,Denver Nuggets,Denver,Colorado,1976
6,Washington Wizards,Washington,District of Columbia,1961
7,Miami Heat,Miami,Florida,1988
8,Orlando Magic,Orlando,Florida,1989
9,Atlanta Hawks,Atlanta,Georgia,1949


---

## Part 6: LIMIT - Controlling Output Size

In [57]:
# Get first 5 teams
query = """
SELECT full_name, city, state
FROM teams
LIMIT 5
"""

first_five = pd.read_sql(query, conn)
print("First 5 Teams:")
display(first_five)

# LIMIT goes at the very end of your query

First 5 Teams:


,full_name,city,state
0,Atlanta Hawks,Atlanta,Georgia
1,Boston Celtics,Boston,Massachusetts
2,Cleveland Cavaliers,Cleveland,Ohio
3,New Orleans Pelicans,New Orleans,Louisiana
4,Chicago Bulls,Chicago,Illinois


In [59]:
# Top 5 oldest teams
query = """
SELECT full_name, city, year_founded
FROM teams
ORDER BY year_founded
LIMIT 5
"""

oldest_five = pd.read_sql(query, conn)
print("Top 5 Oldest NBA Teams:")
display(oldest_five)

# ORDER BY happens first, THEN LIMIT picks the first N

Top 5 Oldest NBA Teams:


,full_name,city,year_founded
0,Boston Celtics,Boston,1946
1,Golden State Warriors,San Francisco,1946
2,New York Knicks,New York,1946
3,Los Angeles Lakers,Los Angeles,1948
4,Sacramento Kings,Sacramento,1948


### Key Points about LIMIT:

- Always goes at the end of your query
- Useful for testing queries on large datasets
- **pandas equivalent:** `df.head(5)`
- Combine with ORDER BY for "Top N" queries

---

## Part 7: Working with Game Data

In [63]:
# Let's peek at the team_game_stats table structure
query = "PRAGMA table_info(team_game_stats)"
game_structure = pd.read_sql(query, conn)
print("Team Game Stats Columns:")
display(game_structure)

Team Game Stats Columns:


,cid,name,type,notnull,dflt_value,pk
0,0,season,TEXT,0,None,1
1,1,game_id,TEXT,0,None,2
2,2,team_id,INTEGER,0,None,3
3,3,game_date,TEXT,0,None,0
4,4,matchup,TEXT,0,None,0
5,5,wl,TEXT,0,None,0
6,6,pts,INTEGER,0,None,0
7,7,fgm,INTEGER,0,None,0
8,8,fga,INTEGER,0,None,0
9,9,fg3m,INTEGER,0,None,0


In [65]:
# Get a sample of game data
query = """
SELECT season, game_id, team_id, game_date, pts, wl
FROM team_game_stats
LIMIT 10
"""

sample_games = pd.read_sql(query, conn)
print("Sample Game Data:")
display(sample_games)

Sample Game Data:


,season,game_id,team_id,game_date,pts,wl
0,2021-22,0022100002,1610612744,2021-10-19,121,W
1,2021-22,0022100001,1610612751,2021-10-19,104,L
2,2021-22,0022100002,1610612747,2021-10-19,114,L
3,2021-22,0022100001,1610612749,2021-10-19,127,W
4,2021-22,0022100007,1610612739,2021-10-20,121,L
5,2021-22,0022100009,1610612740,2021-10-20,97,L
6,2021-22,0022100012,1610612743,2021-10-20,110,W
7,2021-22,0022100008,1610612745,2021-10-20,106,L
8,2021-22,0022100010,1610612753,2021-10-20,97,L
9,2021-22,0022100012,1610612756,2021-10-20,98,L


In [67]:
# Find high-scoring games (120+ points) from 2021-22 season
query = """
SELECT game_date, team_id, pts, wl
FROM team_game_stats
WHERE season = '2021-22' AND pts >= 120
ORDER BY pts DESC
LIMIT 10
"""

high_scoring = pd.read_sql(query, conn)
print("Top 10 Highest-Scoring Team Performances (2021-22):")
display(high_scoring)

print(f"\nHighest score: {high_scoring['pts'].max()} points")

Top 10 Highest-Scoring Team Performances (2021-22):


,game_date,team_id,pts,wl
0,2022-01-26,1610612766,158,W
1,2022-02-25,1610612759,157,W
2,2022-02-25,1610612764,153,L
3,2022-04-01,1610612746,153,W
4,2021-12-02,1610612763,152,W
5,2022-03-15,1610612751,150,W
6,2022-03-14,1610612750,149,W
7,2021-11-27,1610612745,146,W
8,2022-04-10,1610612747,146,W
9,2021-12-04,1610612738,145,W



Highest score: 158 points


### Why This Matters for ML:

Instead of loading ALL games into memory:
```python
# BAD - loads everything!
df = pd.read_csv('all_nba_games.csv')  # Maybe 100,000 rows
df = df[(df['season'] == '2021-22') & (df['pts'] >= 120)]
```

We use SQL to filter FIRST:
```python
# GOOD - only loads what we need!
query = "SELECT * FROM games WHERE season = '2021-22' AND pts >= 120"
df = pd.read_sql(query, conn)  # Maybe only 200 rows
```

**This is how real data scientists work with large datasets**

---

## Part 8: Putting It All Together - Complete Queries

In [72]:
# Complex query: Find all wins by teams that scored 100-110 points in 2021-22
query = """
SELECT game_date, team_id, pts, wl
FROM team_game_stats
WHERE season = '2021-22'
  AND pts BETWEEN 100 AND 110
  AND wl = 'W'
ORDER BY pts DESC
LIMIT 20
"""

moderate_wins = pd.read_sql(query, conn)
print("Wins with 100-110 Points (2021-22):")
display(moderate_wins)

print(f"\nTotal games found: {len(moderate_wins)}")

Wins with 100-110 Points (2021-22):


,game_date,team_id,pts,wl
0,2021-10-20,1610612743,110,W
1,2021-10-22,1610612762,110,W
2,2021-10-24,1610612753,110,W
3,2021-10-27,1610612758,110,W
4,2021-10-28,1610612755,110,W
5,2021-10-29,1610612761,110,W
6,2021-10-30,1610612765,110,W
7,2021-11-05,1610612757,110,W
8,2021-11-09,1610612762,110,W
9,2021-11-17,1610612737,110,W



Total games found: 20


### BETWEEN Operator:

- `WHERE pts BETWEEN 100 AND 110` is the same as `WHERE pts >= 100 AND pts <= 110`
- More readable for range queries
- Includes both endpoints (100 and 110)

In [82]:
# Query for ML feature engineering
# Get all game stats we might need for a win prediction model
query = """
SELECT 
    team_id,
    pts,
    fgm,
    fga,
    fg3m,
    fg3a,
    reb,
    ast,
    stl,
    blk,
    tov,
    wl
FROM team_game_stats
WHERE season = '2021-22'
LIMIT 100
"""

ml_ready_data = pd.read_sql(query, conn)
print("Data Ready for Machine Learning:")
display(ml_ready_data.head(10))

print(f"\nShape: {ml_ready_data.shape}")
print(f"Features: {list(ml_ready_data.columns)}")
print(f"\nWin rate in this sample: {(ml_ready_data['wl'] == 'W').mean():.2%}")

Data Ready for Machine Learning:


,team_id,pts,fgm,fga,fg3m,fg3a,reb,ast,stl,blk,tov,wl
0,1610612749,127,48,105,17,45,54,25,8,9,8,W
1,1610612751,104,37,84,17,32,44,19,3,9,13,L
2,1610612744,121,41,93,14,39,50,30,9,2,17,W
3,1610612747,114,45,95,15,42,45,21,7,4,18,L
4,1610612754,122,42,90,17,47,51,29,2,10,17,L
5,1610612766,123,46,107,13,31,46,29,9,5,8,W
6,1610612741,94,37,86,7,23,48,18,8,5,17,W
7,1610612765,88,36,90,6,28,47,17,7,5,17,L
8,1610612738,134,48,117,21,57,56,34,13,9,18,L
9,1610612752,138,51,105,17,45,55,27,9,10,19,W



Shape: (100, 12)
Features: ['team_id', 'pts', 'fgm', 'fga', 'fg3m', 'fg3a', 'reb', 'ast', 'stl', 'blk', 'tov', 'wl']

Win rate in this sample: 50.00%


### This DataFrame is Ready for ML!

Next steps would be:
1. Encode `wl` as binary (W=1, L=0)
2. Split into X (features) and y (target)
3. Train/test split
4. Train decision tree model
5. Predict game outcomes!

### **Why is this DataFrame ready for ML?!**

To predict points, we treat points as the outcome and use other stats as inputs.
To predict wins, we use game stats to learn which combinations usually lead to a win.
For clustering, we give the model only stats and let it group teams with similar play styles.

Refresh on Decision Trees: A decision tree can predict whether a team will win or lose using game statistics.
The tree learns a series of yes/no questions (like points or turnovers) that best
separate wins from losses. Each path through the tree explains the prediction.



---

## Part 9: Query Structure Summary

### Complete SQL Query Template:

```sql
SELECT column1, column2, column3    -- What columns to show
FROM table_name                     -- Which table to use
WHERE condition                     -- Filter rows (optional)
ORDER BY column                     -- Sort results (optional)
LIMIT number                        -- Limit results (optional)
```

### Order MATTERS!

1. SELECT (what to show)
2. FROM (which table)
3. WHERE (filter rows)
4. ORDER BY (sort)
5. LIMIT (limit results)

**You cannot put LIMIT before ORDER BY!**

---

## Part 10: SQL vs. Pandas Comparison

### Same Operations, Different Syntax:

| Operation | Pandas | SQL |
|-----------|--------|-----|
| Load all data | `pd.read_csv('file.csv')` | `SELECT * FROM table` |
| Select columns | `df[['col1', 'col2']]` | `SELECT col1, col2 FROM table` |
| Filter rows | `df[df['pts'] > 100]` | `WHERE pts > 100` |
| Sort | `df.sort_values('pts')` | `ORDER BY pts` |
| Sort descending | `df.sort_values('pts', ascending=False)` | `ORDER BY pts DESC` |
| First N rows | `df.head(10)` | `LIMIT 10` |
| Multiple conditions (AND) | `df[(df['pts'] > 100) & (df['wl'] == 'W')]` | `WHERE pts > 100 AND wl = 'W'` |
| Multiple conditions (OR) | `df[(df['state'] == 'CA') | (df['state'] == 'TX')]` | `WHERE state = 'CA' OR state = 'TX'` |

### Key Differences:

- SQL uses `=` for comparison (pandas uses `==`)
- SQL uses `AND` / `OR` (pandas uses `&` / `|`)
- SQL text needs single quotes: `'California'`
- SQL is declarative ("what you want"), pandas is procedural ("how to get it")

---

## Summary & Next Steps

In [93]:
# Always close your database connection when done!
conn.close()
print("✅ Database connection closed")

✅ Database connection closed


### What You Learned Today:

✅ **Connect to SQLite databases**
✅ **SELECT** - Choose columns to display
✅ **FROM** - Specify which table
✅ **WHERE** - Filter rows based on conditions
✅ **ORDER BY** - Sort results (ASC/DESC)
✅ **LIMIT** - Control output size
✅ **Load SQL results into pandas** for ML

### SQL Query Pattern:

```python
# 1. Write SQL query as a string
query = """
SELECT columns
FROM table
WHERE conditions
ORDER BY column
LIMIT n
"""

# 2. Execute query and load into DataFrame
df = pd.read_sql(query, conn)

# 3. Now use pandas/sklearn for ML!
```

### Next Lesson:

- **Aggregate functions:** COUNT, SUM, AVG, MIN, MAX
- **GROUP BY:** Create summary statistics
- **Feature engineering** using SQL
- Build ML model using NBA data!

### Practice:

Complete **ai04a_Tasks.ipynb** to practice writing your own queries!